# **Resolvedor de Sudoku**


Nesse case irei utilizar o OpenCV e Keras para montar um resolvedor automático de Sudoku. Primeiro irei criar uma rede neural que reconhece dígitos e treiná-la utilizando o dataset de dígitos, Mnist do Keras. Então, irei utilizar o openCV para extrair os dígitos de uma imagem de sudoku e passá-los para a rede neural treinada. Por fim, a biblioteca 'sudoku' irá formatar os números num tabuleiro, e em seguida resolvê-lo.

**Bibliotecas utilizadas**

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import imutils
from sudoku import Sudoku
from skimage.segmentation import clear_border
from imutils.perspective import four_point_transform
from keras.preprocessing.image import img_to_array
from keras.models import load_model

**Funções utilizadas**

In [2]:
def exibe_image(img, gray=True):
    plt.figure(figsize=(10, 20))
    if gray:
        plt.imshow(img, cmap = 'gray')
    else:
        plt.imshow(img)
    
    plt.axis('off')
    plt.show()

In [3]:
def find_puzzle(image_path):
    # leitura e transformação da imagem
    img = cv2.imread(image_path)
    img_cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(img_cinza, (7, 7), 3)

    # threshold
    img_thr = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2) 
    img_inv = cv2.bitwise_not(img_thr)

    # encontrando os contornos do puzzle
    contours = cv2.findContours(img_inv.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    puzzle_contours = None
    for contour in contours:
        perimetro = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * perimetro, True)

        if len(approx) == 4:
            puzzle_contours = approx
            break
    
    # cisalhamento
    warped = four_point_transform(img_cinza, puzzle_contours.reshape(4, 2))
    
    return warped

In [4]:
def resize_image(image):
    img_resized = cv2.resize(image, (28, 28))
    img_resized = np.array(img_resized).astype('float') / 255
    img_resized = img_to_array(img_resized)
    img_resized = np.expand_dims(img_resized, axis=0)
    
    return img_resized

In [5]:
def extract_digit(image, model):
    # threshold
    thresh = cv2.threshold(image, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY_INV)[1]
    thresh = clear_border(thresh)
    (h, w) = thresh.shape

    # econtrando os contornos do dígito
    contour = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = imutils.grab_contours(contour)

    if len(contour) == 0:
        return 0
        
    else:
        c = max(contour, key = cv2.contourArea)
        
        # centralizando o dígito na imagem
        x, y, larg, alt = cv2.boundingRect(c)
        esquerda = ((w - larg) // 2) - x
        cima = ((h - alt) // 2) - y
        deslocamento = np.float32([[1, 0, esquerda], [0, 1, cima]])
        thresh = cv2.warpAffine(thresh, deslocamento, (w, h))

        # reencontrando os contornos na imagem centralizada
        contour = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contour = imutils.grab_contours(contour)
        
        c = max(contour, key = cv2.contourArea)

        mask = np.zeros(thresh.shape, dtype = 'uint8')
    
        cv2.drawContours(mask, [c], -1, 255, -1)
    
        percent_filled = cv2.countNonZero(mask) / float(w * h)

        if percent_filled < 0.03:
            return 0

        thresh_copy = thresh.copy() 
        resized_image = resize_image(thresh_copy)

        # predição do dígito
        pred = model.predict(np.array(resized_image))
        pred_label = np.argmax(pred, axis=1)
        digit = pred_label[0] 

    return digit

In [6]:
def create_sudoku_board(image_warped, model):
    # criando a matriz do sudoku
    step_x = image_warped.shape[1] // 9
    step_y = image_warped.shape[0] // 9

    sudoku_matrix = []
    for y in range(9):
        row = []
        for x in range(9):
            start_y = y * step_y
            end_y = (y + 1) * step_y

            start_x = x * step_x
            end_x = (x + 1) * step_x

            number = extract_digit(image_warped[start_y:end_y, start_x:end_x], model)
            
            row.append(number)
            
        sudoku_matrix.append(row)

    # criando o tabuleiro do sudoku
    sudoku_board = Sudoku(3, 3, board=sudoku_matrix)
    
    return sudoku_board

In [8]:
model_sudoku = load_model('sudoku_model.h5')

**Teste em imagens de sudoku**

Imagem 1

<img src="./imagens_sudokus/sudoku.png" alt="Imagem 1" style="height: 408px; width: 360px"/>

In [9]:
puzzle_warped = find_puzzle('./imagens_sudokus/sudoku.png')
board = create_sudoku_board(puzzle_warped, model_sudoku)

print('Sudoku digital')
board.show()

solution = board.solve()

print('Solução')
solution.show()

Sudoku digital
+-------+-------+-------+
| 8     |   1   |     9 |
|   5   | 8   7 |   1   |
|     4 |   9   | 7     |
+-------+-------+-------+
|   6   | 7   1 |   2   |
| 5   8 |   6   | 1   7 |
|   1   | 5   2 |   9   |
+-------+-------+-------+
|     7 |   4   | 6     |
|   8   | 3   9 |   4   |
| 3     |   5   |     8 |
+-------+-------+-------+

Solução
+-------+-------+-------+
| 8 7 2 | 4 1 3 | 5 6 9 |
| 9 5 6 | 8 2 7 | 3 1 4 |
| 1 3 4 | 6 9 5 | 7 8 2 |
+-------+-------+-------+
| 4 6 9 | 7 3 1 | 8 2 5 |
| 5 2 8 | 9 6 4 | 1 3 7 |
| 7 1 3 | 5 8 2 | 4 9 6 |
+-------+-------+-------+
| 2 9 7 | 1 4 8 | 6 5 3 |
| 6 8 5 | 3 7 9 | 2 4 1 |
| 3 4 1 | 2 5 6 | 9 7 8 |
+-------+-------+-------+



Imagem 2

<img src="./imagens_sudokus/sudoku_ex.png" alt="Imagem 2" style="height: 280px; width: 390px"/>

In [10]:
puzzle_warped = find_puzzle('./imagens_sudokus/sudoku_ex.png')
board = create_sudoku_board(puzzle_warped, model_sudoku)

print('Sudoku digital')
board.show()

solution = board.solve()

print('Solução')
solution.show()

Sudoku digital
+-------+-------+-------+
| 5 3   |   7   |       |
| 6     | 1 9 5 |       |
|   9 8 |       |   6   |
+-------+-------+-------+
| 8     |   6   |     3 |
| 4     | 8   3 |     1 |
| 7     |   2   |     6 |
+-------+-------+-------+
|   6   |       | 2 8   |
|       | 4 1 9 |     5 |
|       |   8   |   7 9 |
+-------+-------+-------+

Solução
+-------+-------+-------+
| 5 3 4 | 6 7 8 | 9 1 2 |
| 6 7 2 | 1 9 5 | 3 4 8 |
| 1 9 8 | 3 4 2 | 5 6 7 |
+-------+-------+-------+
| 8 5 9 | 7 6 1 | 4 2 3 |
| 4 2 6 | 8 5 3 | 7 9 1 |
| 7 1 3 | 9 2 4 | 8 5 6 |
+-------+-------+-------+
| 9 6 1 | 5 3 7 | 2 8 4 |
| 2 8 7 | 4 1 9 | 6 3 5 |
| 3 4 5 | 2 8 6 | 1 7 9 |
+-------+-------+-------+



Imagem 3

<img src="./imagens_sudokus/sudoku_ex11.jpg" alt="Imagem 2" style="height: 380px; width: 504px"/>

In [11]:
puzzle_warped = find_puzzle('./imagens_sudokus/sudoku_ex11.jpg')
board = create_sudoku_board(puzzle_warped, model_sudoku)

print('Sudoku digital')
board.show()

solution = board.solve()

print('Solução')
solution.show()

Sudoku digital
+-------+-------+-------+
|       | 2 3   |       |
|   6 7 |       | 9 2   |
|   9   |     7 |   3   |
+-------+-------+-------+
|     4 |   7   |     8 |
| 6     | 4   2 |     1 |
| 7     |   1   | 6     |
+-------+-------+-------+
|   7   | 6     |   1   |
|   1 8 |       | 3 7   |
|       |   5 1 |       |
+-------+-------+-------+

Solução
+-------+-------+-------+
| 8 4 5 | 2 3 9 | 1 6 7 |
| 3 6 7 | 1 4 8 | 9 2 5 |
| 2 9 1 | 5 6 7 | 8 3 4 |
+-------+-------+-------+
| 1 5 4 | 3 7 6 | 2 9 8 |
| 6 8 3 | 4 9 2 | 7 5 1 |
| 7 2 9 | 8 1 5 | 6 4 3 |
+-------+-------+-------+
| 4 7 2 | 6 8 3 | 5 1 9 |
| 5 1 8 | 9 2 4 | 3 7 6 |
| 9 3 6 | 7 5 1 | 4 8 2 |
+-------+-------+-------+



Imagem 4

<img src="./imagens_sudokus/sudoku_ex7.png" alt="Imagem 2" style="height: 352px; width: 352px"/>

In [12]:
puzzle_warped = find_puzzle('./imagens_sudokus/sudoku_ex7.png')
board = create_sudoku_board(puzzle_warped, model_sudoku)

print('Sudoku digital')
board.show()

solution = board.solve()

print('Solução')
solution.show()

Sudoku digital
+-------+-------+-------+
|       |       |       |
|   4 3 |       | 8 5   |
| 5     | 2   6 |     3 |
+-------+-------+-------+
| 6     |   1   |     9 |
|   7   |       |   4   |
|     8 |       | 7     |
+-------+-------+-------+
|       | 9   1 |       |
|       |   2   |       |
|       |       |       |
+-------+-------+-------+

Solução
+-------+-------+-------+
| 1 6 9 | 3 5 8 | 2 7 4 |
| 2 4 3 | 1 7 9 | 8 5 6 |
| 5 8 7 | 2 4 6 | 1 9 3 |
+-------+-------+-------+
| 6 2 4 | 5 1 7 | 3 8 9 |
| 3 7 1 | 8 9 2 | 6 4 5 |
| 9 5 8 | 4 6 3 | 7 1 2 |
+-------+-------+-------+
| 4 3 2 | 9 8 1 | 5 6 7 |
| 7 1 5 | 6 2 4 | 9 3 8 |
| 8 9 6 | 7 3 5 | 4 2 1 |
+-------+-------+-------+



Imagem 5

<img src="./imagens_sudokus/sudoku_ex5.jpg" alt="Imagem 5" style="height: 320px; width: 276px"/>

In [13]:
puzzle_warped = find_puzzle('./imagens_sudokus/sudoku_ex5.jpg')
board = create_sudoku_board(puzzle_warped, model_sudoku)

print('Sudoku digital')
board.show()

solution = board.solve()

print('Solução')
solution.show()

Sudoku digital
+-------+-------+-------+
| 3     |   9   |   5   |
|     9 |   2   | 8     |
|   6   | 4     | 1     |
+-------+-------+-------+
|     3 |     6 |   8 9 |
|       | 2 8 9 |       |
| 6 9   | 3     | 5     |
+-------+-------+-------+
|     5 |     3 |   1   |
|     1 |   7   | 2     |
|   4   |   5   |     3 |
+-------+-------+-------+

Solução
+-------+-------+-------+
| 3 8 4 | 7 9 1 | 6 5 2 |
| 7 1 9 | 6 2 5 | 8 3 4 |
| 5 6 2 | 4 3 8 | 1 9 7 |
+-------+-------+-------+
| 4 2 3 | 5 1 6 | 7 8 9 |
| 1 5 7 | 2 8 9 | 3 4 6 |
| 6 9 8 | 3 4 7 | 5 2 1 |
+-------+-------+-------+
| 2 7 5 | 9 6 3 | 4 1 8 |
| 9 3 1 | 8 7 4 | 2 6 5 |
| 8 4 6 | 1 5 2 | 9 7 3 |
+-------+-------+-------+



**Falha no reconhecimento do sudoku**

No exemplo abaixo, o algoritmo de OCR não foi capaz de encontrar todos os dígitos da imagem. Após um pouco de investigação, descobri que um pequeno ajuste em como a foto foi tirada resolveria: a parte de cima da folha possui algumas dobras, o que atrapalha o cisalhamento.

<img src="./imagens_sudokus/sudoku_ex8.jpg" alt="Imagem 2" style="height: 210px; width: 210px"/>

In [14]:
puzzle_warped = find_puzzle('./imagens_sudokus/sudoku_ex8.jpg')
board = create_sudoku_board(puzzle_warped, model_sudoku)

print('Sudoku digital')
board.show()

Sudoku digital
+-------+-------+-------+
|       |       |       |
| 7   6 |       |     9 |
|       |       |       |
+-------+-------+-------+
|   7   |       |       |
| 8     |       |       |
| 4     |       |       |
+-------+-------+-------+
|       |       |       |
|       |       |       |
|       |       |       |
+-------+-------+-------+



# **Conclusão**

O meu objetivo com esse projeto era criar um algoritmo generalista que reconhecesse o tabuleiro de sudoku numa imagem. Para isso, utilizei a "Imagem 1" como referência para criação do resolvedor. Dessa forma, a depender da qualidade da imagem, o algoritmo pode falhar, como foi visto anteriormente. 

Assim, concluo que o algoritmo serviu para uma boa quantidade de imagens, cumprindo o seu caráter generalista.